In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def obtener_urls(busqueda, max_urls=10):
    """Obtiene URLs de recetas de AllRecipes para una palabra clave específica."""
    opciones = webdriver.ChromeOptions()
    opciones.add_argument("--headless")  # Ejecutar en modo headless para mayor eficiencia
    driver = webdriver.Chrome(options=opciones)

    urls = []

    try:
        driver.get(f"https://www.allrecipes.com/search?q={busqueda}")
        wait = WebDriverWait(driver, 1)
        resultados = wait.until(EC.presence_of_all_elements_located(
            (By.XPATH, "//a[contains(@class, 'mntl-card-list-card--extendable')]")
        ))

        urls = [resultado.get_attribute("href") for resultado in resultados[:max_urls]]

    except Exception as e:
        print(f"Error al obtener URLs para '{busqueda}': {e}")
    
    finally:
        driver.quit()

    return urls


def recolectar_recetas(max_total=500, max_por_busqueda=10):
    """Recolecta URLs de recetas de AllRecipes hasta alcanzar el total deseado."""
    palabras_clave = [
        "bread", "chicken", "cake", "pasta", "salad", "soup", "cookies", "dessert",
        "pizza", "beef", "vegetarian", "vegan", "gluten-free", "breakfast", "seafood",
        "sandwich", "burger", "smoothie", "pancakes", "muffins", "pie", "appetizer",
        "snacks", "rice", "steak", "meatballs", "tacos", "burritos", "cheese", "potatoes",
        "pork", "fish", "spaghetti", "lasagna", "curry", "sushi", "roast", "stew",
        "brownies", "ice cream", "wings", "chili", "eggs", "quiche", "casserole",
        "noodles", "keto", "low-carb", "whole30", "instant pot", "slow cooker", "grill"
    ]
    
    recetas_urls = []
    palabras_usadas = []

    for palabra in palabras_clave:
        if len(recetas_urls) >= max_total:
            break
        
        urls_obtenidas = obtener_urls(palabra, max_urls=max_por_busqueda)
        palabras_usadas.append(palabra)
        recetas_urls.extend(urls_obtenidas)

        print(f"Palabra clave: {palabra} | URLs obtenidas: {len(urls_obtenidas)} | Total acumulado: {len(recetas_urls)}")

    recetas_urls = list(set(recetas_urls))  # Eliminar URLs duplicadas
    return recetas_urls[:max_total]


recetas = recolectar_recetas(max_total=500, max_por_busqueda=20)

print(f"Total de URLs recolectadas: {len(recetas)}")
for idx, url in enumerate(recetas, start=1):
    print(f"{idx}. {url}")


Palabra clave: bread | URLs obtenidas: 20 | Total acumulado: 20
Palabra clave: chicken | URLs obtenidas: 20 | Total acumulado: 40
Palabra clave: cake | URLs obtenidas: 20 | Total acumulado: 60
Palabra clave: pasta | URLs obtenidas: 20 | Total acumulado: 80
Palabra clave: salad | URLs obtenidas: 20 | Total acumulado: 100
Palabra clave: soup | URLs obtenidas: 20 | Total acumulado: 120
Palabra clave: cookies | URLs obtenidas: 20 | Total acumulado: 140
Palabra clave: dessert | URLs obtenidas: 20 | Total acumulado: 160
Palabra clave: pizza | URLs obtenidas: 20 | Total acumulado: 180
Palabra clave: beef | URLs obtenidas: 20 | Total acumulado: 200
Palabra clave: vegetarian | URLs obtenidas: 20 | Total acumulado: 220
Palabra clave: vegan | URLs obtenidas: 20 | Total acumulado: 240
Palabra clave: gluten-free | URLs obtenidas: 20 | Total acumulado: 260
Palabra clave: breakfast | URLs obtenidas: 20 | Total acumulado: 280
Palabra clave: seafood | URLs obtenidas: 20 | Total acumulado: 300
Palabra c

In [3]:
print(recetas)

['https://www.allrecipes.com/recipe/155135/favorite-bourbon-pecan-pie/', 'https://www.allrecipes.com/recipe/16972/sausage-applesauce-appetizer/', 'https://www.allrecipes.com/recipe/33176/delicious-gluten-free-pancakes/', 'https://www.allrecipes.com/recipe/44839/chris-bay-area-burger/', 'https://www.allrecipes.com/recipe/20697/breakfast-pizza-i/', 'https://www.allrecipes.com/recipe/25483/sensational-sirloin-kabobs/', 'https://www.allrecipes.com/recipe/100606/beef-bulgogi/', 'https://www.allrecipes.com/recipe/152175/vegan-agave-cornbread-muffins/', 'https://www.allrecipes.com/recipe/144059/jumbo-breakfast-cookies/', 'https://www.allrecipes.com/recipe/24094/butter-flaky-pie-crust/', 'https://www.allrecipes.com/recipe/6864/monkey-bread-ii/', 'https://www.allrecipes.com/recipe/9827/chocolate-chocolate-chip-cookies-i/', 'https://www.allrecipes.com/recipe/257657/lemon-ricotta-pancakes/', 'https://www.allrecipes.com/bang-bang-blended-chicken-burger-recipe-7370938', 'https://www.allrecipes.com/

In [4]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from concurrent.futures import ThreadPoolExecutor, as_completed
import time


def obtener_receta_info(url):
    """Obtiene información detallada de una receta desde una URL."""
    opciones = webdriver.ChromeOptions()
    opciones.add_argument("--headless")  # Modo headless para eficiencia
    driver = webdriver.Chrome(options=opciones)

    receta_info = {}
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 1)

        # Obtener el título de la receta
        titulo = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//h1[contains(@class, 'headline')]")
        )).text

        # Obtener los ingredientes
        try:
            ingredientes = driver.find_elements(By.CSS_SELECTOR, "li.mm-recipes-structured-ingredients__list-item")
            ingredientes_lista = [ingrediente.text for ingrediente in ingredientes if ingrediente.text.strip()]
        except:
            ingredientes_lista = ["Ingredientes no encontrados"]


        # Obtener el contenido restante (instrucciones de preparación)
        instrucciones = driver.find_elements(By.CSS_SELECTOR, "li.comp.mntl-sc-block.mntl-sc-block-startgroup.mntl-sc-block-group--LI")
        instrucciones_lista = [instruccion.text for instruccion in instrucciones]

        # Crear el diccionario con la información
        receta_info = {
            "ID": url.split("/")[-2],  # Usar el ID de la URL
            "Nombre": titulo,
            "Ingredientes": "; ".join(ingredientes_lista),
            "Instrucciones": " ".join(instrucciones_lista),
            "URL": url
        }

    except Exception as e:
        print(f"Error al procesar la URL {url}: {e}")

    finally:
        driver.quit()

    return receta_info


def guardar_csv(datos, nombre_archivo):
    """Guarda los datos de las recetas en un archivo CSV."""
    with open(nombre_archivo, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["ID", "Nombre", "Ingredientes", "Instrucciones", "URL"])
        writer.writeheader()
        writer.writerows(datos)


def recolectar_datos_recetas(urls, nombre_archivo):
    """Recolecta información de recetas desde una lista de URLs y guarda un CSV."""
    recetas = []

    print(f"Procesando {len(urls)} recetas...\n")  # Mensaje inicial

    # Usar ThreadPoolExecutor para ejecutar las solicitudes en paralelo
    with ThreadPoolExecutor(max_workers=5) as executor:  # Ajusta max_workers según el rendimiento de tu máquina
        future_to_url = {executor.submit(obtener_receta_info, url): url for url in urls}
        for i, future in enumerate(as_completed(future_to_url), 1):
            url = future_to_url[future]
            try:
                receta_info = future.result()
                if receta_info:
                    recetas.append(receta_info)
                    print(f"[{i}/{len(urls)}] Receta procesada: {receta_info['Nombre']}")
            except Exception as e:
                print(f"Error al procesar la URL {url}: {e}")

    guardar_csv(recetas, nombre_archivo)
    print(f"\nArchivo CSV creado: {nombre_archivo}")


recolectar_datos_recetas(recetas, "recetas.csv")


Procesando 492 recetas...

[1/492] Receta procesada: Chris' Bay Area Burger
[2/492] Receta procesada: Sausage Applesauce Appetizer
[3/492] Receta procesada: Best Breakfast Pizza
[4/492] Receta procesada: Delicious Gluten-Free Pancakes
[5/492] Receta procesada: Favorite Bourbon Pecan Pie
[6/492] Receta procesada: Sensational Steak Kabobs
[7/492] Receta procesada: Beef Bulgogi
[8/492] Receta procesada: Jumbo Breakfast Cookies
[9/492] Receta procesada: Vegan Agave Cornbread Muffins
[10/492] Receta procesada: Butter Flaky Pie Crust
[11/492] Receta procesada: Bread Machine Monkey Bread
[12/492] Receta procesada: Lemon Ricotta Pancakes
[13/492] Receta procesada: Bang Bang Blended Chicken Burger
[14/492] Receta procesada: Chocolate Chocolate Chip Cookies
[15/492] Receta procesada: Sweet Potato Casserole Dessert
[16/492] Receta procesada: Pumpkin Chocolate Chip Muffins
[17/492] Receta procesada: Beef Stroganoff with Ground Beef
[18/492] Receta procesada: Vegan Broccoli Soup
[19/492] Receta pro

In [ ]:
import pandas as pd

# Ruta del archivo CSV existente
ruta_csv = "recetas.csv"

# Recolectar recetas (asegúrate de haber ejecutado tu función `recolectar_recetas` antes)
recetas = recolectar_recetas(max_total=8, max_por_busqueda=20)  # Limitar a 8 recetas

# Crear un DataFrame para las nuevas recetas
nuevas_recetas = []
for idx, url in enumerate(recetas, start=1):
    receta_info = {
        "ID": range(1, len(recetas) + 1),  # ID extraído de la URL
        "Nombre": f"Receta {idx}",  # Nombre temporal
        "Ingredientes": "Ingredientes no disponibles",
        "Instrucciones": "Instrucciones no disponibles",
        "URL": url
    }
    nuevas_recetas.append(receta_info)

df_nuevas = pd.DataFrame(nuevas_recetas)

# Cargar el archivo CSV existente
df_existente = pd.read_csv(ruta_csv)

# Concatenar las nuevas recetas al archivo existente
df_actualizado = pd.concat([df_existente, df_nuevas], ignore_index=True)

# Actualizar la columna 'ID' para mantener números consecutivos
df_actualizado['ID'] = range(1, len(df_actualizado) + 1)

# Guardar el DataFrame actualizado en el archivo CSV
df_actualizado.to_csv(ruta_csv, index=False)

print("¡Se han agregado 8 nuevas filas al archivo CSV!")
